In [1]:
from pndapetzim.data import get_labeled_data
from pndapetzim.data import ORDER_FILE_NAME
from pndapetzim.data import LABEL_FILE_NAME

df = get_labeled_data(
    '../data/' + ORDER_FILE_NAME,
    '../data/' + LABEL_FILE_NAME,
)

In [ ]:
len(set(df.restaurant_id))